In [20]:
##RAG Pipelines- Data ingestion to vector DB pipeline

import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [21]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 1 PDF files to process

Processing: HA201 - SAP HANA 2.0 SPS05 - High Availability and Disaster Tolerance Administration.pdf
  ✓ Loaded 204 pages

Total documents loaded: 204


In [22]:
all_pdf_documents

[Document(metadata={'producer': 'PDFKit', 'creator': 'PDFKit', 'creationdate': '2021-04-28T17:10:09+00:00', 'source': '../data/pdf/HA201 - SAP HANA 2.0 SPS05 - High Availability and Disaster Tolerance Administration.pdf', 'total_pages': 204, 'page': 0, 'page_label': '1', 'source_file': 'HA201 - SAP HANA 2.0 SPS05 - High Availability and Disaster Tolerance Administration.pdf', 'file_type': 'pdf'}, page_content='HA201\nSAP HANA 2.0 SPS05 - High\nAvailability and Disaster Tolerance\nAdministration\n.\n.\nPARTICIPANT HANDBOOK\nINSTRUCTOR-LED TRAINING\n.\nCourse Version: 17\nCourse Duration: 3 Day(s)\ne-book Duration: 7 Hours 5 Minutes\nMaterial Number: 50155431'),
 Document(metadata={'producer': 'PDFKit', 'creator': 'PDFKit', 'creationdate': '2021-04-28T17:10:09+00:00', 'source': '../data/pdf/HA201 - SAP HANA 2.0 SPS05 - High Availability and Disaster Tolerance Administration.pdf', 'total_pages': 204, 'page': 1, 'page_label': '2', 'source_file': 'HA201 - SAP HANA 2.0 SPS05 - High Availabil

In [23]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=800,chunk_overlap=100):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [24]:
chunks=split_documents(all_pdf_documents)
chunks

Split 204 documents into 509 chunks

Example chunk:
Content: HA201
SAP HANA 2.0 SPS05 - High
Availability and Disaster Tolerance
Administration
.
.
PARTICIPANT HANDBOOK
INSTRUCTOR-LED TRAINING
.
Course Version: 17
Course Duration: 3 Day(s)
e-book Duration: 7 Ho...
Metadata: {'producer': 'PDFKit', 'creator': 'PDFKit', 'creationdate': '2021-04-28T17:10:09+00:00', 'source': '../data/pdf/HA201 - SAP HANA 2.0 SPS05 - High Availability and Disaster Tolerance Administration.pdf', 'total_pages': 204, 'page': 0, 'page_label': '1', 'source_file': 'HA201 - SAP HANA 2.0 SPS05 - High Availability and Disaster Tolerance Administration.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'PDFKit', 'creator': 'PDFKit', 'creationdate': '2021-04-28T17:10:09+00:00', 'source': '../data/pdf/HA201 - SAP HANA 2.0 SPS05 - High Availability and Disaster Tolerance Administration.pdf', 'total_pages': 204, 'page': 0, 'page_label': '1', 'source_file': 'HA201 - SAP HANA 2.0 SPS05 - High Availability and Disaster Tolerance Administration.pdf', 'file_type': 'pdf'}, page_content='HA201\nSAP HANA 2.0 SPS05 - High\nAvailability and Disaster Tolerance\nAdministration\n.\n.\nPARTICIPANT HANDBOOK\nINSTRUCTOR-LED TRAINING\n.\nCourse Version: 17\nCourse Duration: 3 Day(s)\ne-book Duration: 7 Hours 5 Minutes\nMaterial Number: 50155431'),
 Document(metadata={'producer': 'PDFKit', 'creator': 'PDFKit', 'creationdate': '2021-04-28T17:10:09+00:00', 'source': '../data/pdf/HA201 - SAP HANA 2.0 SPS05 - High Availability and Disaster Tolerance Administration.pdf', 'total_pages': 204, 'page': 2, 'page_label': '3', 'source_file': 'HA201 - SAP HANA 2.0 SPS05 - High Availabil

In [25]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [27]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            import os
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 509


In [28]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 509 texts...


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Generated embeddings with shape: (509, 384)
Adding 509 documents to vector store...
Successfully added 509 documents to vector store
Total documents in collection: 1018


In [29]:
chunks

[Document(metadata={'producer': 'PDFKit', 'creator': 'PDFKit', 'creationdate': '2021-04-28T17:10:09+00:00', 'source': '../data/pdf/HA201 - SAP HANA 2.0 SPS05 - High Availability and Disaster Tolerance Administration.pdf', 'total_pages': 204, 'page': 0, 'page_label': '1', 'source_file': 'HA201 - SAP HANA 2.0 SPS05 - High Availability and Disaster Tolerance Administration.pdf', 'file_type': 'pdf'}, page_content='HA201\nSAP HANA 2.0 SPS05 - High\nAvailability and Disaster Tolerance\nAdministration\n.\n.\nPARTICIPANT HANDBOOK\nINSTRUCTOR-LED TRAINING\n.\nCourse Version: 17\nCourse Duration: 3 Day(s)\ne-book Duration: 7 Hours 5 Minutes\nMaterial Number: 50155431'),
 Document(metadata={'producer': 'PDFKit', 'creator': 'PDFKit', 'creationdate': '2021-04-28T17:10:09+00:00', 'source': '../data/pdf/HA201 - SAP HANA 2.0 SPS05 - High Availability and Disaster Tolerance Administration.pdf', 'total_pages': 204, 'page': 2, 'page_label': '3', 'source_file': 'HA201 - SAP HANA 2.0 SPS05 - High Availabil

In [30]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [31]:
rag_retriever

In [32]:
rag_retriever.retrieve("Can you explain what is zero downtime?")

Retrieving documents for query: 'Can you explain what is zero downtime?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_ca327bb5_15',
  'content': 'That means less than an hour of downtime per year. This can be a very challenging target. To\nmeet such challenging targets, high availability and disaster tolerance should be an integral\npart of the architectural design, that is, implemented on every layer of the infrastructure.\nDowntime is the consequence of outages, which may be planned downtime (such as that for\nsystem upgrades or hardware replacements) or caused by unplanned downtime (such as that\nfor software or hardware failures). Unplanned downtime can be triggered by equipment\nmalfunction, software, or network failures, or a major disaster such as a fire, earthquake, a\nregional power loss, or a construction accident which may decommission the entire data\ncenter.',
  'metadata': {'doc_index': 15,
   'file_type': 'pdf',
   'creator': 'PDFKit',
   'source_file': 'HA201 - SAP HANA 2.0 SPS05 - High Availability and Disaster Tolerance Administration.pdf',
   'producer': 'PDFKit',
   'p

In [33]:
###RAG Pipeline- VectorDB To LLM Output Generation

import os
from dotenv import load_dotenv
load_dotenv()

print(os.getenv("OPENAI_API_KEY"))

None


In [36]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage

ModuleNotFoundError: No module named 'langchain.prompts'

In [ ]:
class OpenAILLM:
    def __init__(self, model_name: str = "gpt-4o-mini", api_key: str = None):
        """
        Initialize OpenAI LLM
        
        Args:
            model_name: OpenAI model name
            api_key: OpenAI API key (or set OPENAI_API_KEY env var)
        """
        self.model_name = model_name
        self.api_key = api_key or os.getenv("OPENAI_API_KEY")

        if not self.api_key:
            raise ValueError("OPENAI_API_KEY is required")

        self.client = OpenAI(api_key=self.api_key)
        print(f"Initialized OpenAI LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str) -> str:
        """
        Generate response using retrieved context (RAG-safe)
        """
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use ONLY the context below to answer.

Context:
{context}

Question:
{question}

Answer:"""
        )

        prompt = prompt_template.format(
            context=context,
            question=query
        )

        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1,
                max_tokens=1024
            )
            return response.choices[0].message.content

        except Exception as e:
            return f"Error generating response: {e}"

    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation
        """
        prompt = f"""Context:
{context}

Question:
{query}

Answer:"""

        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1,
                max_tokens=1024
            )
            return response.choices[0].message.content

        except Exception as e:
            return f"Error: {e}"
